<a href="https://colab.research.google.com/github/adelic-matf/UMMP/blob/main/UMMP_lab_6_Gurobi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Softveri i biblioteke za rešavanje problema linearnog programiranja

Za rešavanje problema linearnog programiranja, postoji mnogo softvera koji se koriste u akademske i industrijske svrhe. Najpoznatiji su:
- Gurobi
- CPLEX
- Lingo
- GLPK (GNU Linear Programming Kit) - otvorenog koda

Takođe, u Python-u postoje mnoge biblioteke za rešavanje optimizacionih problema. Neke od njih su:
- PuLP
- Pyomo
- cvxpy

Ne ovom kurstu upoznaćemo se sa radom u Gurobi softveru, koji je jedan od najbržih i najefikasnijih rešavača za linearno i celobrojno programiranje.
Gurobi takođe može rešavati probleme sa kvadratnim funkcijama cilja i/ili kvadratnim ograničenjima, što je deo nelinearnih problema, ali generalno ne rešava probleme nelinearne optimizacije.


 Iako je komercijalan, Gurobi nudi mogućnost korišćenja besplatnih licenci za različite potrebe:

- [Akademska licenca](https://www.gurobi.com/free-trial/) (besplatna za studente, nastavnike, istraživače) - obično nema ograničenja u broju promenljivih i ograničenja, samo zahteva validnu registraciju i dobijanje licence.
- Size-Limited Trial License — Instalacija Gurobi paketa putem pip install gurobipy u Colab-u automatski uključuje ovu besplatnu licencu. Može rešavati modele sa najviše 2000 promenljivih i 2000 ograničenja. Ova licenca je namenjena samo za istraživanje i razvoj. Nije dozvoljeno korišćenje u komercijalne svrhe.


# Instalacija Gurobi-ja

In [1]:
!pip install gurobipy  # install gurobipy, if not already installed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 101.0 MB/s eta 0:00:00


## Učitavanje paketa i kreiranje optimizacionog modela

Učitajmo upravo insatlirani paket:

In [2]:
import gurobipy as gp


Paket gurobipy sadrži klasu pod nazivom [Model](https://docs.gurobi.com/projects/optimizer/en/current/reference/python/model.html). Pokretanjem komande:

In [3]:
prvigp_model = gp.Model()

Restricted license - for non-production use only - expires 2026-11-23


kreira se objekat pod nazivom prvigp_model koji predstavlja jedan optimizacioni problem. To je osnovna klasa za definisanje i rešavanje optimizacionih modela u Gurobiju. Ona omogućava kreiranje promenljivih, dodavanje ograničenja i funkcije cilja, rešavanje problema i analiziranje rezultata.


Osim gurobipy paketa učitaćemo i [GRB modul](https://docs.gurobi.com/projects/optimizer/en/current/reference/python/grb.html):

In [4]:
from gurobipy import GRB

 koji sadrži različite konstante koje nam koriste za definisanje modela:

- tipove promenljivih (GRB.BINARY, GRB.CONTINUOUS, GRB.INTEGER, ...)

- tipove funkcije cilja (GRB.MINIMIZE, GRB.MAXIMIZE)

- statusne kodove (GRB.OPTIMAL, GRB.INFEASIBLE, ...)

- parametre koji se mogu menjati (GRB.Param.TimeLimit, GRB.Param.MIPGap, ...)

## Definisanje promenljvih

Za definisanje optimizacionih promenljivih objekta koji smo kreirali imamo na raspolaganju sledeće metode klase Model:
- addVar() dodaje jednu promenljivu
- addVars() za dodavanje većeg broja promenljivih

Metod addVar ima sledeći oblik:

**addVar(lb=0.0, ub=float('inf'), obj=0.0, vtype=GRB.CONTINUOUS, name='', column=None)**

Svi argumenti imaju podrazumevane vrednosti, pa se metoda može pozvati i bez navođenja bilo kog argumenta.

Ipak, razjasnimo šta svaki od njih predstavlja:

- lb - donja granica nove promenljive;

- ub - gornja granica nove promenljive;

- obj - koeficijent nove promenljive u funkciji cilja;

- vtype - tip optimizacione promenljive (npr. GRB.CONTINUOUS, GRB.BINARY, GRB.INTEGER).

- name - naziv promenljive (koristan za ispis i identifikaciju).

- column - objekat tipa Column koji određuje u kojim ograničenjima učestvuje nova promenljiva i sa kojim koeficijentima.

Na primer, koristeći komande

In [5]:
x = prvigp_model.addVar(name="x1")
y = prvigp_model.addVar(name="x2")

dodajemo dve optimizacione promenljive u model.

Argument name="x1" i name="x2" daje naziv tim promenljivama (što olakšava pregled i debagovanje modela kada ga kad ga ispišmo ili eksportujemo).

Dakle:

- x postaje Gurobi promenljiva sa imenom "x1";

- y postaje Gurobi promenljiva sa imenom "x2".

Kreirane promenljive x i y sada mogu učestvovati u ograničenjima i funkciji cilja modela. Po defaultu, one su kontinualne (GRB.CONTINUOUS), nenegativne (lb=0.0) i sa koeficijentom 0 u funkciji cilja (ako se ne navede drugačije). Dakle, ne moramo unositi ograničenje nenegativnosti promenljivih, ali ako optimizaciona promenljiva nema ograničenja, to moramo eksplicitno naglasiti:

In [6]:
y = prvigp_model.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY, name="x2")

Drugi metod **addVars()** se koristi kada želimo definisati veći broj promenljivih. Na primer

In [7]:
n=4; #broj optimizacionih promenljivih
x = prvigp_model.addVars(n, name="x")

Sada je x rečnik sa ključevima od 0 do n-1. Svakoj promenljivoj pristupamo preko $x[i]$, a u ispisu rezultata njeno ime će takođe biti $x[i]$. Proverimo i tip nove promenljive

In [8]:
type(x)

gurobipy._core.tupledict

Ako želimo da su ključevi indeksirani od 1 do n  onda možemo uraditi sledeće

In [9]:
x = prvigp_model.addVars(range(1,n+1), name="x")

Za dalji rad u gurbipy iskoristimo zadatak, koji je rešen i u skripti

$$
\begin{array}
 \text{(min)}\quad &-2x_1+x_3\\
 \\
  \text{p.o.} \quad  &x_1+x_2+x_3+x_4= 1\\
     &x_2+2x_3+x_4= 2\\
     &-3x_1-x_2+x_3-x_4= 1\\
     &x_1,x_2,x_3,x_4\geq 0.
    \end{array}
  $$

Sobzirom na to da smo u prethodnim koracima, dodali mnogo optimizacionih promenljivih koje nam ne trebaju za ovaj konkretan primer, definisaćemo ponovo model. Tako ćemo dobiti prazan objekat, pa ćemo ponovo definisati optimizacione promenljive.




In [10]:
prvigp_model = gp.Model()
x = prvigp_model.addVars(range(1,n+1), name="x")

## Definisanje funkcije cilja

Funkciju cilja definišemo pomoću metoda **setObjective()**:

In [11]:
prvigp_model.setObjective(-2*x[1]+x[3], GRB.MINIMIZE)

Zadatak koji rešavamo je problem minimizacije, što smo definisali navođenjem konstante GRB.MINIMIZE.

Promenljive koje koristimo u zapisu funkcije cilja moraju biti prethodno definisane kao promenljive modela, inače će rašavač prijaviti grešku.

Jedan model može imati samo jednu funkciju cilja, tj. ako pozovemo setObjective više puta, poslednji poziv će zameniti prethodni funkciju cilja.

Funkcija cilja može biti izražena i preko izraza sa sumama, vektorima.
Na primer, mogli smo je definisati i sa:

In [12]:
c = [-2, 0, 1, 0] #lista koeficijenata funkcije cilja, indeksiranje ide od 0, za razliku od rečnika promenljivih x koje smo ranije definisali
prvigp_model.setObjective(gp.quicksum(c[i-1] * x[i] for i in range(1,n+1)), GRB.MINIMIZE)

## Dodavanje ograničenja

Objektu klase Model ograničenja dodajemo metodom addConstr(). Gurobi interno prevodi ograničenja u standardni oblik koji rešavač koristi. Ograničenja mogu biti nejednakosti (<=, >=) i jednakosti (==). Imena ograničenja (name) nisu obavezna, ali mogu biti korisna kada se analiziraju rezultati.

In [13]:
prvigp_model.addConstr(x[1] + x[2]+x[3]+x[4] == 1, name="ogr1")
prvigp_model.addConstr(x[2]+2*x[3]+x[4]==2, name="ogr2")
prvigp_model.addConstr(-3*x[1] - x[2]+x[3]-x[4] == 1, name="ogr3")

<gurobi.Constr *Awaiting Model Update*>

Za dodavanje ograničenja možemo kosrititi i metodu **addConstrs()** ako želimo dodati više njih u petlji.

## Rešavanje LP problema

Konačno, kada smo završili sa postavljanjem svih informacija u objektu, možemo ga rešiti koristeći metodu optimize():

In [14]:
prvigp_model.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 3 rows, 4 columns and 11 nonzeros
Model fingerprint: 0x966f2af4
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 3 rows and 4 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.000000000e+00


Analizirajmo informacije koje smo dobili. Prvo su navedene informacije o verziji Gurobi-ja koja se koristi i sistemu na kome se rešava problem:
- Verzija Gurobi-ja: 12.0.2
- OS: Ubuntu 22.04.4 LTS
- CPU: Intel Xeon, koristi SIMD instrukcije (SSE2, AVX, AVX2)
- Thread count: koristi do 2 paralelna niti, iako CPU ima 1 fizičko jezgro i 2 logička (hyperthreading).



Zatim slede informacije o zadatku koji se rešava:
*Optimize a model with 3 rows, 4 columns and 11 nonzeros*.

Model ima:
- 3 jednačine
- 4 promenljive
- 11 nenula elemenata u matrici A

Model fingerprint: 0xa1c57cbf
Hash vrednost modela (broj u heksadecimalnom formatu) i služi za identifikaciju modela.  Ako napravimo i najmanju promenu u modelu (npr. promenimo koeficijent u jednom ograničenju), vrednost fingerprinta će se promeniti.


Posle toga sledi karatka statistika o podacima:
- Matrix range: najmanji i najveći koeficijenti u A
- Objective range: najmanji i najveći koeficijenti u funkciji cilja
- Bounds range: svi imaju iste granice (0)
- RHS range: desne strane ograničenja (od 1 do 2)

Dalje slede informacije o pripremnoj fazi modela. U okviru ove faze, rešavač proverava sistem jednačina uklanja očigledno redundantne ili rešenjem određene promenljive i ograničenja.

U ovom slučaju ceo model je rešiv u presolve fazi, što znači da se optimalno rešenje može dobiti bez pokretanja iterativnog (simpleks) algoritma.

Nakon toga sledi tabela sa informacijama:
- Iteration - u prvoj i jedinoj iteraciji je rešen problem
- Objective - optimalna vrednost je jednaka 1
- Primal Inf. = 0.0 - sve ograničenja zadovoljena
- Dual Inf. = 0.0 - dualni uslovi optimalnosti takođe zadovoljeni
- Time - vreme za koje je rešen problem

## Prikazivanje rezultata i čuvanje podataka




Najjendostavniji način da prikažemo rezutlate jeste direktnim štampanjem u terminalu. Metod getVars() vraća listu svih promenljivih u modelu. Atributi promenljivih su:
- VarName - ime promenljive
- X - optimalna vrednost promenljive (dostupna samo nako što je problem rešen)
- LB -donja granica promenljive
- UB - gornja granica promenljive
- Obj - odgovarajući koeficijent u funkciji cilja

i još neke koje nećemo koristiti.

Odštampajmo rezultate u terminali koristeći ove informacije:

In [15]:
if prvigp_model.status == GRB.OPTIMAL:
    print("Optimalna vrednost funkcije cilja:", prvigp_model.ObjVal)
    print("Vrednosti promenljivih su:")
    for x in prvigp_model.getVars():
        print(f"{x.VarName} = {x.X}")
else:
    print("Model nije uspešno rešen.")

Optimalna vrednost funkcije cilja: 1.0
Vrednosti promenljivih su:
x[1] = 0.0
x[2] = 0.0
x[3] = 1.0
x[4] = 0.0


Model i rezultate možemo upisati i u fajl. Uobičajeni način za čuvanje informacija o modelu je tekstualni fajl sa ekstenzijom .lp. Format je standardan za sve rešavače, pa ga mogu čitati i drugi softveri.

In [16]:
prvigp_model.write("prvigp_model.lp")

Metod write će kreirati tekstulani fajl prvigp_model.lp i u njega upisati funkciju cilja i jednačine ograničanja na način koji je čoveku jednostavno čitljiv. Na Google Colabu, ovako kreiran fajl, možemo pronaći na levoj strani prozora, otvaranjem ikonice za folder.

Za tekstualni fajl sa rešenjima problema, Gurobi koristi svoj poseban format koji nije standardizovan, ali je jedostavan i čitljiv. Takav fajl dobijamo korišćenjem ekstenzije .sol.

In [17]:
prvigp_model.write("prvigp_model_rez.sol")

In [18]:
p = prvigp_model.presolve()
p.write("prvigp_model_presolve.lp")


Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Presolve a model with 3 rows, 4 columns and 11 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 3 rows and 4 columns


In [19]:
prvigp_model.setParam("LogFile","resavanje.log")

Set parameter LogFile to value "resavanje.log"


# Drugi primer

In [20]:
model2=gp.Model()

In [21]:
n=4
x = model2.addVars(range(1,n+1), name="x")

In [22]:
model2.setObjective(2*x[1]-x[2]+2*x[3]+x[4], GRB.MAXIMIZE)

In [23]:
model2.addConstr(x[1] +x[2]+2*x[3]-x[4]>= 9, name="ogr1")
model2.addConstr(x[1]+x[2] +x[3] <=10, name="ogr2")
model2.addConstr(x[2]+x[3]+x[4]  <= 9, name="ogr3")

<gurobi.Constr *Awaiting Model Update*>

In [24]:
model2.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 3 rows, 4 columns and 10 nonzeros
Model fingerprint: 0x339f2fc1
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+00, 1e+01]
Presolve time: 0.01s
Presolved: 3 rows, 4 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0000000e+30   4.000000e+30   5.000000e+00      0s
       4    2.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.500000000e+01
